# Pip install section

In [1]:
pip install imbalanced-learn

# Import

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split


from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

from imblearn.over_sampling import RandomOverSampler 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN

from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.calibration import CalibratedClassifierCV

from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import RandomUnderSampler

from xgboost import XGBClassifier
from scipy.stats import uniform, randint

from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN

import itertools


import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
import pickle

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import cv2
import numpy as np
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#load data

first test our model is put on gpu

In [5]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

my already preprocessed and scaled data is saved in google drive, you can upload it to google colab and change the below file path to acess it

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
qt_scaled_list =       pickle.load( open( 'drive/MyDrive/colab Data/JIT/pickle/qt_scaled.p', "rb" ) )
openstack_scaled_list= pickle.load( open( 'drive/MyDrive/colab Data/JIT/pickle/openstack_scaled.p', "rb" ) )
Xy_scaled_shuffled_list= pickle.load( open( 'drive/MyDrive/colab Data/JIT/pickle/Xy_scaled_shuffled.p', "rb" ) )


qt

In [8]:
Xq_train_scaled=qt_scaled_list[0]
yq_train=qt_scaled_list[1]
Xq_test_scaled=qt_scaled_list[2]
yq_test=qt_scaled_list[3]
print(Xq_train_scaled.shape)
print(yq_train.shape)
print(Xq_test_scaled.shape)
print(yq_test.shape)

(19056, 17)
(19056,)
(4765, 17)
(4765,)


openstack

In [9]:
Xo_train_scaled=openstack_scaled_list[0]
yo_train=openstack_scaled_list[1]
Xo_test_scaled=openstack_scaled_list[2]
yo_test=openstack_scaled_list[3]
print(Xo_train_scaled.shape)
print(yo_train.shape)
print(Xo_test_scaled.shape)
print(yo_test.shape)

(9632, 17)
(9632,)
(2409, 17)
(2409,)


XY

In [10]:
X_train_scaled=Xy_scaled_shuffled_list[0]
y_train=Xy_scaled_shuffled_list[1]
X_test_scaled=Xy_scaled_shuffled_list[2]
y_test=Xy_scaled_shuffled_list[3]
print(X_train_scaled.shape)
print(y_train.shape)
print(X_test_scaled.shape)
print(y_test.shape)

(28688, 17)
(28688,)
(7174, 17)
(7174,)


# define result prinint method

define two printing methods

In [11]:
def printRes(name,ytest,ypred):
    print(name)
    print('recall',   round(recall_score(   ytest, ypred, pos_label=1),3))
    print('precision',round(precision_score(ytest, ypred, pos_label=1),3))
    print('f1',       round(f1_score(       ytest, ypred, pos_label=1),3))
    print('auc',      round(roc_auc_score(  ytest, ypred),3))
    print()
    

In [12]:
trainDataPrintList=[X_test_scaled,Xq_test_scaled,Xo_test_scaled]
testDataPrintList=[y_test,yq_test,yo_test]
dataNamePrintList=['all','qt','openstack']
metricPrintList=['recall','precision','f1','auc']
def getRes(curModel):
    
    recallList=[]
    precisionList=[]
    f1List=[]
    aucList=[]
    
    for i in range(3):
        ypred=curModel.predict(trainDataPrintList[i])
        
        recallList.append(   round(recall_score(   testDataPrintList[i], ypred, pos_label=1),3))
        precisionList.append(round(precision_score(testDataPrintList[i], ypred, pos_label=1),3))
        f1List.append(       round(f1_score(       testDataPrintList[i], ypred, pos_label=1),3))
        aucList.append(      round(roc_auc_score(  testDataPrintList[i], ypred),3))
        
    tempdf = pd.DataFrame([recallList,precisionList,f1List,aucList], index =metricPrintList,columns=dataNamePrintList).T
    return tempdf

# Try XGBoost first to make sure everything is ok with the basic result

first do oversampling using SMOTE

In [13]:
rs = SMOTE(random_state=42)
X_train_scaled_uo,y_train_uo = rs.fit_resample(X_train_scaled,y_train)
print(X_train_scaled_uo.shape)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(51742, 17)


In [14]:
XGB = XGBClassifier(objective="binary:logistic", random_state=42,use_label_encoder=False,eval_metric='auc',
                   scale_pos_weight=2,
                   )
XGB.fit(X_train_scaled_uo, y_train_uo)
curModel=XGB

tempdf=getRes(curModel)
tempdf



,recall,precision,f1,auc
all,0.692,0.212,0.324,0.689
qt,0.594,0.184,0.281,0.671
openstack,0.804,0.242,0.372,0.679


# ANN with oversampling (strongly overfitting)

In [18]:
rs = SMOTE(random_state=42)
X_train_scaled_uo,y_train_uo = rs.fit_resample(X_train_scaled,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


convert each number into float type

In [19]:
X_train=np.copy(X_train_scaled_uo).astype(np.float32)
y_train=np.copy(y_train_uo)
X_test=np.copy(X_test_scaled).astype(np.float32)
y_test=np.copy(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(51742, 17)
(51742,)
(7174, 17)
(7174,)


convert train and test data into tensor

In [20]:
X_train = torch.tensor(X_train)
y_train=  torch.tensor(y_train)
X_test=  torch.tensor(X_test)
y_test=  torch.tensor(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([51742, 17])
torch.Size([51742])
torch.Size([7174, 17])
torch.Size([7174])


speicfy gpu device

In [21]:
device = torch.device("cuda:0")

define ANN structure

In [22]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        
        self.hiddenNum=100

        self.fc1 = nn.Linear(17, self.hiddenNum)                #fully connected layer
        self.bn1 = nn.BatchNorm1d(num_features=self.hiddenNum)  #batch normalization
        self.drop1= nn.Dropout(0.2)                             #dropout
        self.fc10 = nn.Linear(self.hiddenNum, 2)                #2 classes

    def forward(self, x):
        
        x=torch.relu(self.bn1(self.fc1(x)))
        x=self.drop1(x)
        x = self.fc10(x)


        return torch.softmax(x, dim=1)


get prediciton method

In [29]:
def get_pred(testData):
  y_pred=[]
  net.eval()
  with torch.no_grad():
    for i in (range(len(testData))):
        tempX=testData[i].view(-1,17).to(device)
        net_out=net(tempX)[0]        
        predicted_class=torch.argmax(net_out).cpu()
        y_pred.append(predicted_class)
  return y_pred

convert test data to tensor data

In [24]:
Xq_test_ann=torch.tensor(Xq_test_scaled.astype(np.float32))
Xo_test_ann=torch.tensor(Xo_test_scaled.astype(np.float32))

train the ann and get the evaluation result

In [30]:
net=Net().to(device)

loss_function=nn.CrossEntropyLoss()
optimizer=optim.AdamW(net.parameters(),lr=0.001,weight_decay=1e-3)

BATCH_SIZE=64
EPOCHS=5

for epoch in range(EPOCHS):
    for i in (range(0,len(X_train),BATCH_SIZE)):
        
        batch_x=X_train[i:i+BATCH_SIZE].view(-1,17)
        batch_y=y_train[i:i+BATCH_SIZE]
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        net.zero_grad()
        outputs=net(batch_x)   
        loss=loss_function(outputs,batch_y)
      
        loss.backward()
        optimizer.step()
    print('epoch',epoch,'loss:',loss)
    print()


y_pred=get_pred(X_test)
printRes('all',y_test,y_pred)

y_pred=get_pred(Xq_test_ann)
printRes('qt',yq_test,y_pred)
    
y_pred=get_pred(Xo_test_ann)
printRes('openstack',yo_test,y_pred)

epoch 0 loss: tensor(0.3140, device='cuda:0', grad_fn=<NllLossBackward>)

epoch 1 loss: tensor(0.3149, device='cuda:0', grad_fn=<NllLossBackward>)

epoch 2 loss: tensor(0.3154, device='cuda:0', grad_fn=<NllLossBackward>)

epoch 3 loss: tensor(0.3157, device='cuda:0', grad_fn=<NllLossBackward>)

epoch 4 loss: tensor(0.3168, device='cuda:0', grad_fn=<NllLossBackward>)

all
recall 1.0
precision 0.109
f1 0.196
auc 0.5

qt
recall 1.0
precision 0.087
f1 0.161
auc 0.5

openstack
recall 1.0
precision 0.151
f1 0.262
auc 0.5



**as we can see, even just after 1 epoch, the auc is 0.5, and the recall is 1, which indicates overfitting to the extreme**



# ANN without oversampling

now we follow the same processing procedure as before, chaning into float type, change into tensor type

In [36]:
X_train_scaled=Xy_scaled_shuffled_list[0]
y_train=Xy_scaled_shuffled_list[1]

X_train_scaled_uo=np.copy(X_train_scaled)
y_train_uo=np.copy(y_train)
print(X_train_scaled_uo.shape)
print(y_train_uo.shape)

(28688, 17)
(28688,)


In [37]:
X_train=np.copy(X_train_scaled_uo).astype(np.float32)
y_train=np.copy(y_train_uo)
X_test=np.copy(X_test_scaled).astype(np.float32)
y_test=np.copy(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(28688, 17)
(28688,)
(7174, 17)
(7174,)


In [38]:
X_train = torch.tensor(X_train)
y_train=  torch.tensor(y_train)
X_test=  torch.tensor(X_test)
y_test=  torch.tensor(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([28688, 17])
torch.Size([28688])
torch.Size([7174, 17])
torch.Size([7174])


In [39]:
device = torch.device("cuda:0")

In [40]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()


        self.hiddenNum=100

        self.fc1 = nn.Linear(17, self.hiddenNum)  
        self.bn1 = nn.BatchNorm1d(num_features=self.hiddenNum)
        self.drop1= nn.Dropout(0.2)
        
        self.fc10 = nn.Linear(self.hiddenNum, 2)  

    def forward(self, x):
        
        x=torch.relu(self.bn1(self.fc1(x)))
        x=self.drop1(x)
        x = self.fc10(x)
        return torch.softmax(x, dim=1)


In [42]:
def get_pred(testData):
  y_pred=[]
  net.eval()
  with torch.no_grad():
    for i in (range(len(testData))):

        tempX=testData[i].view(-1,17).to(device)
        net_out=net(tempX)[0]        
        predicted_class=torch.argmax(net_out).cpu()
        y_pred.append(predicted_class)
  return y_pred

Xq_test_ann=torch.tensor(Xq_test_scaled.astype(np.float32))
Xo_test_ann=torch.tensor(Xo_test_scaled.astype(np.float32))

In [56]:
net=Net().to(device)

loss_function=nn.CrossEntropyLoss(weight=torch.tensor([1.0,10.0])).to(device)
# optimizer=optim.Adam(net.parameters(),lr=0.001,weight_decay=1e-4)
optimizer=optim.AdamW(net.parameters(),lr=0.001,weight_decay=1e-2)
# optimizer=optim.RMSprop(net.parameters(),lr=0.001,weight_decay=1e-3)
# optimizer=optim.SGD(net.parameters(),lr=0.001,weight_decay=1e-4)
# optimizer=optim.Adamax(net.parameters(),lr=0.001,weight_decay=1e-4)
BATCH_SIZE=64
EPOCHS=5

bestLoss=1000000

for epoch in range(EPOCHS):
    curloss=[]
    for i in (range(0,len(X_train),BATCH_SIZE)):
        
        batch_x=X_train[i:i+BATCH_SIZE].view(-1,17)
        batch_y=y_train[i:i+BATCH_SIZE]
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        net.zero_grad()
        outputs=net(batch_x)   
        loss=loss_function(outputs,batch_y)

        if loss < bestLoss: 
          torch.save(net.state_dict(), 'best-model-parameters.pt')

        
        curloss.append(loss.item())

        loss.backward()
        optimizer.step()

    print('epoch',epoch,'loss:',str(round(np.mean(curloss),4)))
    print()



net.load_state_dict(torch.load('best-model-parameters.pt'))
net.to(device)

y_pred=get_pred(X_test)
printRes('all',y_test,y_pred)

y_pred=get_pred(Xq_test_ann)
printRes('qt',yq_test,y_pred)
    
y_pred=get_pred(Xo_test_ann)
printRes('openstack',yo_test,y_pred)

epoch 0 loss: 0.6363

epoch 1 loss: 0.6164

epoch 2 loss: 0.6065

epoch 3 loss: 0.6036

epoch 4 loss: 0.6011

all
recall 0.605
precision 0.233
f1 0.336
auc 0.681

qt
recall 0.486
precision 0.198
f1 0.281
auc 0.649

openstack
recall 0.741
precision 0.269
f1 0.395
auc 0.692



 **we can see the result is much better now, with a auc score of 0.692 for openstack and 0.649 for qt**

# Trying different ANN structure and hyperparameters

In [58]:
Xq_test_ann=torch.tensor(Xq_test_scaled.astype(np.float32))
Xo_test_ann=torch.tensor(Xo_test_scaled.astype(np.float32))

In [97]:
def get_pred(net,testData):
  y_pred=[]
  net.eval()
  with torch.no_grad():
    for i in (range(len(testData))):

        tempX=testData[i].view(-1,17).to(device)
        net_out=net(tempX)[0]        
        predicted_class=torch.argmax(net_out).cpu()
        y_pred.append(predicted_class)
  return y_pred


define funtion to get the auc score, then define a function to get the result based on the "net" object passed in

In [98]:
def getAUC(ytest,ypred):
    return str(round(roc_auc_score( ytest, ypred),3))

def get_results(net):
  aucList=[]

  y_pred=get_pred(net,X_test)
  aucList.append(getAUC(y_test,y_pred))

  y_pred=get_pred(net,Xq_test_ann)
  aucList.append(getAUC(yq_test,y_pred))
      
  y_pred=get_pred(net,Xo_test_ann)
  aucList.append(getAUC(yo_test,y_pred))

  return aucList

create a function to create a Net object for training with sepcified parameters

In [141]:

def getNet(**kwargs):

  hidden_unit=kwargs['hidden_unit']
  dropout=kwargs['dropout']
  activation=kwargs['activation']
  layer=kwargs['layer']
  
  class Net(nn.Module):
      def __init__(self):
          super().__init__()
          self.hiddenNum=hidden_unit         
          self.fc1 = nn.Linear(17, self.hiddenNum)  
          self.bn1 = nn.BatchNorm1d(num_features=self.hiddenNum)
          self.drop1= nn.Dropout(dropout)  
          self.fc2=  nn.Linear(self.hiddenNum,self.hiddenNum)     
          self.fc10 = nn.Linear(self.hiddenNum, 2)
          self.activation=activation
          self.layer=layer

      def forward(self, x):


          x=self.bn1(self.fc1(x))

          if self.activation=='tanh':
            x=torch.tanh(x)
          else:
            x=torch.relu(x)

          x=self.drop1(x)

          if self.layer==2:

            x=torch.relu(self.fc2(x))
            

          x = self.fc10(x)
          return torch.softmax(x, dim=1)

  return Net()

define the actual training function with hyperparameters passed in

In [138]:
def trainNet(**kwargs):

  learning_rate=kwargs['learning_rate']
  weight_decay=kwargs['weight_decay']
  batch_size=kwargs['batch_size']
  epoch=kwargs['epoch']



  net=getNet(**kwargs).to(device)
  net.train()

  
  loss_function=nn.CrossEntropyLoss(weight=torch.tensor([1.0,10.0])).to(device)

  optimizer=optim.AdamW(net.parameters(),lr=learning_rate,weight_decay=weight_decay)
  BATCH_SIZE=batch_size
  EPOCHS=epoch

  for epoch in range(EPOCHS):
      for i in (range(0,len(X_train),BATCH_SIZE)):
          
          batch_x=X_train[i:i+BATCH_SIZE].view(-1,17)
          batch_y=y_train[i:i+BATCH_SIZE]
          batch_x, batch_y = batch_x.to(device), batch_y.to(device)

          net.zero_grad()
          outputs=net(batch_x)   
          loss=loss_function(outputs,batch_y)
          loss.backward()
          optimizer.step()

  return net


define all the hyperparameters, there all 256 combinations

In [139]:
#hyper parameters lists
hidden_unit_list= [100,200]
dropout_list=[0.2,0.4]
activation_list=['relu','tanh']
layer_list=[2,1]
learning_rate_list=[0.01,0.001]
weight_decay_list=[1e-3,1e-4]
batch_size_list=[64,100]
epoch_list=[5,10]

model_para={}

#get all possible combinations of hyper parameters
combo=list(itertools.product(*[
                                hidden_unit_list,
                                dropout_list,
                                activation_list,
                                layer_list,
                                learning_rate_list,
                                weight_decay_list,
                                batch_size_list,
                                epoch_list,
]))
print(len(combo))



256


iterate over each parameter set and save the result

In [143]:

allResParaMap={}
allResList=[]

comboIndex=0;
for paraCombo in combo:

    hidden_unit=paraCombo[0]
    dropout=paraCombo[1]
    activation=paraCombo[2]
    layer=paraCombo[3]
    learning_rate=paraCombo[4]
    weight_decay=paraCombo[5]
    batch_size=paraCombo[6]
    epoch=paraCombo[7]

    
    model_para={  
                'hidden_unit':hidden_unit, 
                'dropout':dropout, 
                'activation':activation, 
                'layer':layer, 
                'learning_rate':learning_rate, 
                'weight_decay':weight_decay, 
                'batch_size':batch_size, 
                'epoch':epoch, 
    }
    
    #record this set of parameter
    allResParaMap[comboIndex]=model_para
    comboIndex+=1
    
    #create the classifier
    net=trainNet(**model_para)
        
    #add to all result
    tempRes=get_results(net)
    print('finished',comboIndex, tempRes)
    allResList.append(tempRes)


allResList

finished 1 ['0.676', '0.644', '0.68']
finished 2 ['0.679', '0.655', '0.68']
finished 3 ['0.672', '0.646', '0.679']
finished 4 ['0.671', '0.644', '0.678']
finished 5 ['0.672', '0.651', '0.663']
finished 6 ['0.67', '0.649', '0.671']
finished 7 ['0.674', '0.652', '0.677']
finished 8 ['0.674', '0.641', '0.688']
finished 9 ['0.686', '0.654', '0.692']
finished 10 ['0.678', '0.653', '0.679']
finished 11 ['0.686', '0.656', '0.694']
finished 12 ['0.677', '0.646', '0.686']
finished 13 ['0.679', '0.653', '0.683']
finished 14 ['0.684', '0.655', '0.687']
finished 15 ['0.684', '0.656', '0.685']
finished 16 ['0.677', '0.649', '0.683']
finished 17 ['0.676', '0.654', '0.676']
finished 18 ['0.677', '0.658', '0.668']
finished 19 ['0.681', '0.662', '0.679']
finished 20 ['0.68', '0.656', '0.683']
finished 21 ['0.682', '0.656', '0.684']
finished 22 ['0.679', '0.661', '0.672']
finished 23 ['0.677', '0.652', '0.676']
finished 24 ['0.677', '0.638', '0.696']
finished 25 ['0.681', '0.65', '0.689']
finished 26 ['

[['0.676', '0.644', '0.68'],
 ['0.679', '0.655', '0.68'],
 ['0.672', '0.646', '0.679'],
 ['0.671', '0.644', '0.678'],
 ['0.672', '0.651', '0.663'],
 ['0.67', '0.649', '0.671'],
 ['0.674', '0.652', '0.677'],
 ['0.674', '0.641', '0.688'],
 ['0.686', '0.654', '0.692'],
 ['0.678', '0.653', '0.679'],
 ['0.686', '0.656', '0.694'],
 ['0.677', '0.646', '0.686'],
 ['0.679', '0.653', '0.683'],
 ['0.684', '0.655', '0.687'],
 ['0.684', '0.656', '0.685'],
 ['0.677', '0.649', '0.683'],
 ['0.676', '0.654', '0.676'],
 ['0.677', '0.658', '0.668'],
 ['0.681', '0.662', '0.679'],
 ['0.68', '0.656', '0.683'],
 ['0.682', '0.656', '0.684'],
 ['0.679', '0.661', '0.672'],
 ['0.677', '0.652', '0.676'],
 ['0.677', '0.638', '0.696'],
 ['0.681', '0.65', '0.689'],
 ['0.683', '0.656', '0.687'],
 ['0.683', '0.656', '0.689'],
 ['0.68', '0.652', '0.687'],
 ['0.678', '0.642', '0.691'],
 ['0.684', '0.651', '0.694'],
 ['0.685', '0.655', '0.696'],
 ['0.679', '0.652', '0.688'],
 ['0.669', '0.652', '0.675'],
 ['0.67', '0.645

save the result, just in case we lost it

In [144]:
import pickle
pickleList=[allResParaMap,allResList]
pickle.dump( pickleList, open( "res.p", "wb" ) )

create a dataframe to present the result

In [145]:
dftest = pd.DataFrame(allResParaMap).T
allList=[]
qtList=[]
openstackList=[]

for l in allResList:
  allList.append(l[0])
  qtList.append(l[1])
  openstackList.append(l[2])

dftest['all']=allList
dftest['qt']=qtList
dftest['openstack']=openstackList



# Top Results 

ordered by the auc socre of openstack, top 10 results



In [149]:
dftest.sort_values(by=['openstack'],inplace=True,ascending=False)
dftest.head(10)

,hidden_unit,dropout,activation,layer,learning_rate,weight_decay,batch_size,epoch,all,qt,openstack
158,200,0.2,relu,1,0.001,0.0001,100,5,0.684,0.652,0.697
30,100,0.2,relu,1,0.001,0.0001,100,5,0.685,0.655,0.696
95,100,0.4,relu,1,0.001,0.0001,100,10,0.679,0.647,0.696
213,200,0.4,relu,1,0.01,0.0001,64,10,0.683,0.651,0.696
23,100,0.2,relu,1,0.01,0.0001,100,10,0.677,0.638,0.696
154,200,0.2,relu,1,0.001,0.001,100,5,0.686,0.654,0.696
217,200,0.4,relu,1,0.001,0.001,64,10,0.684,0.654,0.695
97,100,0.4,tanh,2,0.01,0.001,64,10,0.671,0.628,0.694
93,100,0.4,relu,1,0.001,0.0001,64,10,0.683,0.651,0.694
65,100,0.4,relu,2,0.01,0.001,64,10,0.683,0.645,0.694


ordered by the auc socre of qt, top 10 results


In [150]:
dftest.sort_values(by=['qt'],inplace=True,ascending=False)
dftest.head(10)

,hidden_unit,dropout,activation,layer,learning_rate,weight_decay,batch_size,epoch,all,qt,openstack
81,100,0.4,relu,1,0.01,0.001,64,10,0.687,0.664,0.683
18,100,0.2,relu,1,0.01,0.001,100,5,0.681,0.662,0.679
34,100,0.2,tanh,2,0.01,0.001,100,5,0.679,0.662,0.66
195,200,0.4,relu,2,0.01,0.001,100,10,0.684,0.662,0.69
62,100,0.2,tanh,1,0.001,0.0001,100,5,0.677,0.661,0.68
79,100,0.4,relu,2,0.001,0.0001,100,10,0.686,0.661,0.686
21,100,0.2,relu,1,0.01,0.0001,64,10,0.679,0.661,0.672
144,200,0.2,relu,1,0.01,0.001,64,5,0.678,0.661,0.671
111,100,0.4,tanh,2,0.001,0.0001,100,10,0.686,0.661,0.686
254,200,0.4,tanh,1,0.001,0.0001,100,5,0.676,0.661,0.675


In [151]:
dftest.sort_values(by=['openstack'],inplace=True,ascending=False)
dftest

,hidden_unit,dropout,activation,layer,learning_rate,weight_decay,batch_size,epoch,all,qt,openstack
158,200,0.2,relu,1,0.001,0.0001,100,5,0.684,0.652,0.697
154,200,0.2,relu,1,0.001,0.001,100,5,0.686,0.654,0.696
23,100,0.2,relu,1,0.01,0.0001,100,10,0.677,0.638,0.696
30,100,0.2,relu,1,0.001,0.0001,100,5,0.685,0.655,0.696
95,100,0.4,relu,1,0.001,0.0001,100,10,0.679,0.647,0.696
...,...,...,...,...,...,...,...,...,...,...,...
228,200,0.4,tanh,2,0.01,0.0001,64,5,0.501,0.5,0.502
165,200,0.2,tanh,2,0.01,0.0001,64,10,0.5,0.5,0.5
197,200,0.4,relu,2,0.01,0.0001,64,10,0.5,0.5,0.5
163,200,0.2,tanh,2,0.01,0.001,100,10,0.5,0.5,0.5


In [152]:
dftest.sort_values(by=['qt'],inplace=True,ascending=False)
dftest

,hidden_unit,dropout,activation,layer,learning_rate,weight_decay,batch_size,epoch,all,qt,openstack
81,100,0.4,relu,1,0.01,0.001,64,10,0.687,0.664,0.683
18,100,0.2,relu,1,0.01,0.001,100,5,0.681,0.662,0.679
195,200,0.4,relu,2,0.01,0.001,100,10,0.684,0.662,0.69
34,100,0.2,tanh,2,0.01,0.001,100,5,0.679,0.662,0.66
62,100,0.2,tanh,1,0.001,0.0001,100,5,0.677,0.661,0.68
...,...,...,...,...,...,...,...,...,...,...,...
228,200,0.4,tanh,2,0.01,0.0001,64,5,0.501,0.5,0.502
165,200,0.2,tanh,2,0.01,0.0001,64,10,0.5,0.5,0.5
197,200,0.4,relu,2,0.01,0.0001,64,10,0.5,0.5,0.5
163,200,0.2,tanh,2,0.01,0.001,100,10,0.5,0.5,0.5


# showcase precision is useless

In [163]:
print(len(y_pred))
print(classification_report(yo_test,y_pred))

2409
              precision    recall  f1-score   support

           0       0.93      0.64      0.76      2046
           1       0.27      0.74      0.40       363

    accuracy                           0.66      2409
   macro avg       0.60      0.69      0.58      2409
weighted avg       0.83      0.66      0.71      2409

